In [11]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from loadItems import items_list


In [12]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [13]:
system_message = "You are a courteous and helpful assistant for Saugaat Stores, a grocery store. "
system_message += "Provide clear, accurate answers in no more than one sentence. "
system_message += "If you are unsure of an answer, politely indicate that you do not know."


In [14]:
def chat(message,history):
    messages = [{"role":"system","content":system_message}]
    for human_res,assistant_res in history:
        messages.append({"role":"user","content":human_res})
        messages.append({"role":"assistant","content":assistant_res})
    messages.append({"role":"user","content":message})
    response = openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

C:\Users\Public\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [15]:
print(items_list)

{'basmati rice': '₹120', 'atta': '₹40', 'sugar': '₹45', 'salt': '₹20', 'tur dal': '₹110', 'moong dal': '₹130', 'chana dal': '₹80', 'toor dal': '₹120', 'besan': '₹60', 'maida': '₹35', 'poha': '₹50', 'rava': '₹40', 'sooji': '₹45', 'mustard oil': '₹150', 'sunflower oil': '₹160', 'olive oil': '₹300', 'ghee': '₹280', 'butter': '₹50', 'milk': '₹50', 'curd': '₹40', 'paneer': '₹80', 'eggs': '₹70', 'chicken': '₹220', 'mutton': '₹650', 'fish': '₹300', 'apples': '₹180', 'bananas': '₹60', 'oranges': '₹100', 'mangoes': '₹200', 'grapes': '₹120', 'papaya': '₹40', 'onions': '₹30', 'tomatoes': '₹40', 'potatoes': '₹25', 'garlic': '₹30', 'ginger': '₹35', 'green chillies': '₹15', 'coriander leaves': '₹10', 'spinach': '₹20', 'carrots': '₹40', 'capsicum': '₹70', 'cauliflower': '₹30', 'cabbage': '₹20', 'bitter gourd': '₹50', 'lemon': '₹40', 'green peas': '₹100', 'toothpaste': '₹50', 'shampoo': '₹120', 'soap': '₹30', 'dishwashing liquid': '₹90', 'detergent powder': '₹70', 'toilet cleaner': '₹60'}


In [16]:
def get_item_price(item_name):
    print(f"Tool get_item_price called for {item_name}")
    item = item_name.lower()
    return items_list.get(item,"Item not found")

In [17]:
get_item_price("Cabbage")

Tool get_item_price called for Cabbage


'₹20'

In [18]:
price_function = {
    "name": "get_item_price",
    "description": "Retrieve the price of an item from the grocery store. Use this function whenever you need to know the item's price, for example, when a customer asks 'How much is the cost of tomatoes?'",
    "parameters": {
        "type": "object",
        "properties": {
            "item_name": {
                "type": "string",
                "description": "The name of the item the customer wants to buy."
            }
        },
        "required": ["item_name"],
        "additionalProperties": False
    }
}


In [19]:
tools = [{"type":"function","function":price_function}]

In [30]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response,item_name = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model = MODEL,messages=messages)

    return response.choices[0].message.content

In [31]:
import json

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    item_name = arguments.get('item_name')
    price = get_item_price(item_name)
    
    response = {
        "role": "tool",
        "content": json.dumps({"item_name": item_name, "price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response,item_name


In [32]:
gr.ChatInterface(fn=chat).launch()

C:\Users\Public\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Tool get_item_price called for cabbage
